In [1]:
#All Code Written by Gary Zeri
#Chapman University Computer Science Major, Member of the LaRue CatLab

#Import All Required Packages Here
import ipywidgets as widgets
from IPython.display import clear_output
from diatomicPotentials import extendedRydberg
from RKRClass import RKR


from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

#May be changing later 
from scipy import integrate
import numpy as np
from basisFunctions import harmonicOscillator
from scipy.misc import derivative as ddx

In [2]:
#Declare All Global Variables Here
figure = {
        "data": [],
        "layout":
        {
           "xaxis":{"title":"Bond Distance"},
           "yaxis":{"title":"Energy"},
           "title":{"text":"Potential Energy Curves"}
        },    
    }    

rydberg = extendedRydberg()
rkr = RKR()

In [3]:
def potentialAnalysis():
    basisSize = 15
    u = (1836*1836) / (1836*2)
    
    #W stands for the vibrational frequency of the diattomic molecule
    #Originates from "Diatomic Moleculs According to the Wave Mechanics"
    #By Philip M. Morse
    #Should I use the morse potential to find w?
    w = 0.02#(rydberg.a1 / (2*np.pi)) * pow(2*rydberg.D / u, 0.5)
    
    HO = harmonicOscillator(basisSize, u,  w, abs(rydberg.Re) + 1)    

    S = np.zeros([basisSize, basisSize])
    V = np.zeros([basisSize, basisSize])
    T = np.zeros([basisSize, basisSize])
    
    for i in range(basisSize):
        for j in range(basisSize):
            
            overlapIntegrand = lambda r : HO.basisSet[i](r) * HO.basisSet[j](r)
            S[i,j] = round(integrate.quad(overlapIntegrand, 0, np.inf)[0], 3)
            
            potentialEnergyIntegrand = lambda r : HO.basisSet[i](r) * rydberg.equation(r) * HO.basisSet[j](r)
            
            V[i,j] += round(integrate.quad(potentialEnergyIntegrand, 0, np.inf, limit=250)[0], 3)
            
            kineticEnergyIntegrand = lambda r : HO.basisSet[i](r) * (-1 / (2 * u)) * ddx(HO.basisSet[j], r, n=2, dx=pow(10, -5))
            
            T[i, j] += round(integrate.quad(kineticEnergyIntegrand, 0, np.inf, limit=250)[0], 3)
    
    print("T"*20)
    print(T)
    print("V")
    print(V)
    H = T + V
    
    print(S)
    vibrationalStates, eigenValues = np.linalg.eigh(H)
    
    dx =.05
    X = []
    Y = []
    for x in range(-200, 400):
        X.append(x*dx)
        Y.append(HO.basisSet[5](x*dx))
    
    plot(X, Y, "HO17")
    
   # print(S)
   # print("S!"*50)
   # print(V)
   # print("V!"*50)
   # print(T)
    
    vibrationalEnergy, eigenValues = np.linalg.eig(H)
    
    vibrationalEnergy = sorted(vibrationalEnergy)
    print(vibrationalEnergy)
    
    for index, ve in enumerate(vibrationalEnergy):
        plot([abs(rydberg.Re)], [ve], "VE " + str(index))
    
    iplot(figure)

##########################################################################################

def plot(x, y, name):
    
    figure["data"].append(
         {
            "type":"scatter",
            "x":x,
            "y":y,
            "connectgaps":True,
            "mode":"markers", 
            "name":name,
            #"marker":{"color":"blue"}
        }
    )
    
##########################################################################################    

#builds the Extended-Rydberg fit for the given radius and energy data
def buildPotential(R, E):
        
    print("Building Rydberg Potential Fit")
    rydberg.fitPotential(R, E)
    
    print("Graphing Rydberg Potential")
    ER = rydberg.graphData(0, R[-1] + 4)
    
    plot(ER[0], ER[1], "Extended Rydberg Fit")
        
##########################################################################################    

def loadFile():
    
    global fileLoad
        
    fileLoad = widgets.FileUpload(
        accept = ".txt",
        multiple = True,
        button_style = "info"
    )
        
    display(fileLoad)
    
    #reset graph data if new file is uploaded
    figure["data"] = []
        
    fileLoad.observe(parsePotentialFile, "value")

##########################################################################################    
    
def parsePotentialFile(file):
    
    clear_output()
    display(choice)
    loadFile()
    
    #Remove all unneded data so that only used data is accessible from the file dictionary
    file = file["new"][list(file["new"].keys())[0]]
    
    #check that file name is greater than five characters to ensure that [-4:] 
    #command will not cause an error when checking that file ends in ".txt"
    if(len(file["metadata"]["name"]) <= 5 or file["metadata"]["name"][-4:] != ".txt"):
            print("Warning! " + file["metadata"]["name"] + "' is not of the '.txt' type!")
    else:
        fileData = file["content"].decode("utf-8").split("\n")
                
        #set up empty lists to store bond distance and respective energy
        global R, E
        R = []
        E = []
        
        #use try catch in case that file is incorrectly formatted
        #and to catch the resulting error from parsing an incorrect file
        #assumed file is formatted with two columns of data separated by a space
        #1st column is bond distance, while 2nd column is the energy at that bond distance
        try:
            for lineNumber, line in enumerate(fileData):
                if(line == ""):
                    continue
                    
                line = line.split(" ")               
                
                R.append(float(line[0]))
                E.append(float(line[1]))
        except:
            print("Warning!! Linenumber " + str(lineNumber) + " in the input file was incorrectly formatted!")
            return
        
        print("Potential Energy Surface Data Successfully Parsed!")
        plot(R, E, "Potential Energy Surface From File")
        
        buildPotential(R, E)
        #iplot(figure)
        potentialAnalysis()
          
##########################################################################################

#Handles logic for building potential curve from the diatomic constants
def diatomicConstantsLogic(buttonData):
    
    clear_output()
    displayDiatomicConstantsMenu()
    
    R, E = rkr.graphData(resolution = .1, endPoint=99)
    
    if not any(filter(np.isnan, R)):
        plot(R, E, "RKR Potential")
        buildPotential(R, E)
        iplot(figure)
    else:
        print("RKR Computation Failed!")
        print("Please Ensure Your Diatomic Constant Values Are Correct!")
        print("Or Contact zeri@chapman.edu To Report The Error")
    
##########################################################################################

def displayDiatomicConstantsMenu():
    
    rkrWidgets = rkr.widgetInput()
    
    display(rkrWidgets[0])
    display(rkrWidgets[1])
    display(startButton)
        
    startButton.on_click(diatomicConstantsLogic)
    
##########################################################################################

#Main Function used to exeute main decision logic of the program
def main(buttonData):

    #Refreshes screen to ensure buttons do not pile up on top of each other
    clear_output()
    display(choice)
    
    #check whether to display the load file menu or the get constatnts menu
    if(buttonData["new"] == "Use File"):
        loadFile()
    else:
        global startButton
        startButton = widgets.Button(description = "Start Calculation", button_style="Info")
        
        displayDiatomicConstantsMenu()

In [4]:
#Run Functions with Jupyter Widgets Overlay Here

print("Please choose how you like to set up the Extended-Rydberg Equation:")
choice = widgets.ToggleButtons(
    options = ["Use File", "Use Diatomic Constants"],
    button_style = "info",
    tooltips = ["", "Use Diatomic Constants"],
    value = None,
)

basisSize = widgets.IntText()

display(choice)

choice.observe(main, "value")

Please choose how you like to set up the Extended-Rydberg Equation:


ToggleButtons(button_style='info', options=('Use File', 'Use Diatomic Constants'), tooltips=('', 'Use Diatomic…